In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


In [ ]:
scaler = MinMaxScaler()

In [ ]:
df_train = pd.read_csv('train.csv', delimiter=';', parse_dates=True)
train_set = df_train[['open']].values
scaled_train_set = scaler.fit_transform(train_set)

In [ ]:
X_train = []
y_train = []

In [ ]:
length = len(scaled_train_set) - 1

In [ ]:
for i in range(60, length):
    X_train.append(scaled_train_set[i-60:i, 0])
    y_train.append(scaled_train_set[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
regressor = Sequential()

In [ ]:
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

In [ ]:
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

In [ ]:
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

In [ ]:
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

In [ ]:
regressor.add(Dense(units=1))

In [ ]:
regressor.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
regressor.fit(X_train, y_train, epochs=100, batch_size=32)

In [ ]:
df_test = pd.read_csv('test.csv', delimiter=';', parse_dates=True)
real_prices = df_test[['open']].values

In [ ]:
df_full = pd.concat((df_train['open'], df_test['open']), axis=0)

In [ ]:
inputs = df_full[len(df_full) - len(df_test)-60:].values

In [ ]:
inputs = inputs.reshape(-1, 1)

In [ ]:
inputs = scaler.transform(inputs)

In [ ]:
X_test = []

In [ ]:
length = len(inputs)

In [ ]:
for i in range(60, length):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)

In [ ]:
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
predicted_prices = regressor.predict(X_test)

In [ ]:
predicted_prices = scaler.inverse_transform(predicted_prices)

In [ ]:
import seaborn as sns

In [ ]:
sns.set(style='whitegrid')

In [ ]:
dims = (25, 10)
fig, ax = plt.subplots(figsize=dims)
plt.plot(real_prices, color='red', label='Реальная стоимость акций')
plt.plot(predicted_prices, color='blue', label='Предсказанная стоимость акций')
plt.title('Прогноз стоимости акций Газпрома за период от 02.11.2020 до 23.01.2021')
plt.xlabel('Дни')
plt.ylabel('Цена')
plt.legend()
plt.show()
#plt.savefig('gazprom_predictions.jpg')

In [ ]:
import math
from sklearn.metrics import mean_squared_error
rmse = math.sqrt(mean_squared_error(real_prices, predicted_prices))

In [ ]:
rmse

In [ ]:
df_total = pd.read_csv('gazp_full.csv', delimiter=';', parse_dates=True)
train = df_total[['open']].values
scaled_set = scaler.fit_transform(train)

In [ ]:
X_train = []
y_train = []

In [ ]:
length = len(scaled_set) - 1

In [ ]:
for i in range(60, length):
    X_train.append(scaled_set[i-60:i, 0])
    y_train.append(scaled_set[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
regressor.fit(X_train, y_train, epochs=100, batch_size=32)

In [ ]:
predictions_list = []

In [ ]:
batch = X_train.copy()[-1:].reshape((1, batch.shape[1], 1))

In [ ]:
for i in range(7):
    predictions_list.append(regressor.predict(batch)[0])
    batch = np.append(batch[:, 1:, :], [[predictions_list[i]]], axis=1)

In [ ]:
predictions_list = scaler.inverse_transform(predictions_list)

In [ ]:
predictions_list

In [ ]:
truly = np.append(predicted_prices, predictions_list)

In [ ]:
dims = (25, 10)
fig, ax = plt.subplots(figsize=dims)
plt.plot(real_prices, color='red', label='Реальная стоимость акций')
plt.plot(truly, color='blue', label='Предсказанная стоимость акций')
#plt.plot(predictions_list, color='orange', label='truly')
plt.title('Прогноз стоимости акций Газпрома до 30.01.2021')
plt.xlabel('Дни')
plt.ylabel('Цена')
plt.legend()
plt.show()
plt.savefig('gazprom_predictions_forecast.jpg')

In [ ]:
predictions_list = []

In [ ]:
batch = X_train.copy()[-180:].reshape((180, batch.shape[1], 1))

In [ ]:
for i in range(180):
    predictions_list.append(regressor.predict(batch)[0])
    batch = np.append(batch[:, 1:, :], [[predictions_list[i]]] * 180, axis=1)

In [ ]:
predictions_list = scaler.inverse_transform(predictions_list)

In [ ]:
truly = np.append(predicted_prices, predictions_list)

In [ ]:
dims = (25, 10)
fig, ax = plt.subplots(figsize=dims)
plt.plot(real_prices, color='red', label='Реальная стоимость акций')
plt.plot(truly, color='blue', label='Предсказанная стоимость акций')
#plt.plot(predictions_list, color='orange', label='truly')
plt.title('Прогноз стоимости акций Газпрома за период от 02.11.2020 до 23.01.2021')
plt.xlabel('Дни')
plt.ylabel('Цена')
plt.legend()
plt.show()
#plt.savefig('gazprom_predictions_forecast.jpg')